# Graph Theory: A Complete Tutorial

This notebook provides a comprehensive introduction to graph theory, covering fundamental concepts, representations, and algorithms with practical Python implementations.

## Table of Contents
1. [What is a Graph?](#what-is-a-graph)
2. [Graph Terminology](#terminology)
3. [Graph Representations](#representations)
4. [Breadth-First Search (BFS)](#bfs)
5. [Depth-First Search (DFS)](#dfs)
6. [Shortest Path Algorithms - SPFA](#spfa)
7. [Bellman-Ford Algorithm](#bellman-ford)
8. [Floyd-Warshall Algorithm](#floyd-warshall)
9. [Minimum Spanning Trees (MST)](#mst)
10. [Eulerian Paths and Circuits](#eulerian)
11. [Hamiltonian Paths and Circuits](#hamiltonian)
12. [Knight's Tour Problem](#knights-tour)
13. [Binary Trees](#binary-trees)
14. [Practical Examples](#examples)

## 1. What is a Graph? <a id='what-is-a-graph'></a>

A **graph** is a mathematical structure used to model relationships between objects. It consists of:

- **Vertices (or Nodes)**: The objects or entities
- **Edges**: The connections or relationships between vertices

### Real-World Examples:
- **Social Networks**: People (vertices) connected by friendships (edges)
- **Road Maps**: Cities (vertices) connected by roads (edges)
- **Web Pages**: Pages (vertices) connected by hyperlinks (edges)
- **Computer Networks**: Computers (vertices) connected by cables (edges)

### Types of Graphs:

1. **Directed vs Undirected**
   - **Undirected**: Edges have no direction (friendship: if A is friends with B, then B is friends with A)
   - **Directed**: Edges have direction (following on Twitter: A can follow B without B following A)

2. **Weighted vs Unweighted**
   - **Unweighted**: All edges are equal (simple connections)
   - **Weighted**: Edges have values/costs (distances, costs, capacities)

3. **Connected vs Disconnected**
   - **Connected**: There's a path between any two vertices
   - **Disconnected**: Some vertices cannot reach others

## 2. Graph Terminology <a id='terminology'></a>

- **Vertex/Node**: A point in the graph
- **Edge**: A connection between two vertices
- **Degree**: Number of edges connected to a vertex
- **Path**: A sequence of vertices connected by edges
- **Cycle**: A path that starts and ends at the same vertex
- **Adjacent vertices**: Vertices connected by an edge
- **Neighbor**: An adjacent vertex
- **Distance**: Number of edges in the shortest path between two vertices

## 3. Graph Representations <a id='representations'></a>

There are two main ways to represent graphs in computer programs:

### 3.1 Adjacency List

An **adjacency list** stores a list of neighbors for each vertex. This is memory-efficient for sparse graphs (graphs with relatively few edges).

**Structure**: A dictionary where:
- Keys = vertices
- Values = lists of neighboring vertices

**Example**: For an undirected graph with edges (0-1), (0-2), (1-3)
```python
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0],
    3: [1]
}
```

**Pros**:
- Space efficient: O(V + E) where V = vertices, E = edges
- Fast to iterate through neighbors
- Good for sparse graphs

**Cons**:
- Slower to check if an edge exists between two specific vertices

In [1]:
# Adjacency List Implementation

class GraphList:
    def __init__(self):
        self.adjacency_list = {}
    
    def add_vertex(self, vertex):
        """Add a new vertex to the graph"""
        if vertex not in self.adjacency_list:
            self.adjacency_list[vertex] = []
    
    def add_edge(self, u, v, directed=False):
        """Add an edge between vertices u and v"""
        # Ensure both vertices exist
        self.add_vertex(u)
        self.add_vertex(v)
        
        # Add edge u -> v
        self.adjacency_list[u].append(v)
        
        # For undirected graph, also add v -> u
        if not directed:
            self.adjacency_list[v].append(u)
    
    def display(self):
        """Display the graph"""
        print("Graph (Adjacency List):")
        for vertex, neighbors in self.adjacency_list.items():
            print(f"  {vertex} -> {neighbors}")

# Example usage
g = GraphList()
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.display()

Graph (Adjacency List):
  0 -> [1, 2]
  1 -> [0, 3]
  2 -> [0, 3]
  3 -> [1, 2]


### 3.2 Adjacency Matrix

An **adjacency matrix** is a 2D array where:
- Rows and columns represent vertices
- `matrix[i][j] = 1` if there's an edge from vertex i to vertex j
- `matrix[i][j] = 0` if there's no edge

**Example**: For the same graph as above (4 vertices)
```
    0  1  2  3
0 [ 0  1  1  0 ]
1 [ 1  0  0  1 ]
2 [ 1  0  0  1 ]
3 [ 0  1  1  0 ]
```

**Pros**:
- Fast edge lookup: O(1) to check if edge exists
- Simple to implement
- Good for dense graphs

**Cons**:
- Space inefficient: Always uses O(V²) space
- Slower to iterate through all neighbors

In [2]:
# Adjacency Matrix Implementation

class GraphMatrix:
    def __init__(self, num_vertices):
        self.num_vertices = num_vertices
        self.matrix = [[0] * num_vertices for _ in range(num_vertices)]
    
    def add_edge(self, u, v, directed=False):
        """Add an edge between vertices u and v"""
        if u >= self.num_vertices or v >= self.num_vertices:
            raise ValueError("Vertex index out of bounds")
        
        self.matrix[u][v] = 1
        if not directed:
            self.matrix[v][u] = 1
    
    def display(self):
        """Display the adjacency matrix"""
        print("Graph (Adjacency Matrix):")
        print("  ", " ".join(str(i) for i in range(self.num_vertices)))
        for i, row in enumerate(self.matrix):
            print(f"{i} [{' '.join(str(x) for x in row)}]")

# Example usage
gm = GraphMatrix(4)
gm.add_edge(0, 1)
gm.add_edge(0, 2)
gm.add_edge(1, 3)
gm.add_edge(2, 3)
gm.display()

Graph (Adjacency Matrix):
   0 1 2 3
0 [0 1 1 0]
1 [1 0 0 1]
2 [1 0 0 1]
3 [0 1 1 0]


### Comparison: When to Use Which?

| Criterion | Adjacency List | Adjacency Matrix |
|-----------|---------------|------------------|
| **Space** | O(V + E) | O(V²) |
| **Check if edge exists** | O(degree) | O(1) |
| **Iterate neighbors** | O(degree) | O(V) |
| **Best for** | Sparse graphs | Dense graphs |
| **Add vertex** | Easy | Expensive (resize) |

**Rule of thumb**: Use adjacency list for most real-world graphs (social networks, road maps, etc.)

## 4. Breadth-First Search (BFS) <a id='bfs'></a>

**BFS** explores a graph level by level, starting from a source vertex. It visits all neighbors before moving to the next level.

### How BFS Works:

1. Start at a source vertex and mark it as visited
2. Add it to a queue
3. While the queue is not empty:
   - Remove the first vertex from the queue
   - Visit all its unvisited neighbors
   - Mark them as visited and add them to the queue

### Visual Example:

```
Graph:     0 --- 1
           |     |
           2 --- 3 --- 4
```

BFS from vertex 0:
- Level 0: Visit 0
- Level 1: Visit 1, 2 (neighbors of 0)
- Level 2: Visit 3 (neighbor of 1 and 2)
- Level 3: Visit 4 (neighbor of 3)

**Order**: [0, 1, 2, 3, 4]

### Applications:
- Finding shortest path in unweighted graphs
- Level-order traversal
- Finding connected components
- Web crawling

### Time Complexity: O(V + E)
### Space Complexity: O(V)

In [3]:
from collections import deque
from typing import Dict, List, Set

def bfs(graph: Dict[int, List[int]], start: int) -> List[int]:
    """
    Perform Breadth-First Search starting from a given vertex.
    
    Args:
        graph: Adjacency list representation
        start: Starting vertex
    
    Returns:
        List of vertices in BFS visitation order
    """
    visited = set()
    order = []
    queue = deque([start])
    visited.add(start)
    
    while queue:
        current = queue.popleft()  # Remove from front of queue
        order.append(current)
        
        # Visit all unvisited neighbors
        for neighbor in graph.get(current, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)  # Add to back of queue
    
    return order

# Example graph
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2, 4],
    4: [3]
}

print("Graph:", graph)
print("\nBFS starting from vertex 0:", bfs(graph, 0))
print("BFS starting from vertex 2:", bfs(graph, 2))

Graph: {0: [1, 2], 1: [0, 3], 2: [0, 3], 3: [1, 2, 4], 4: [3]}

BFS starting from vertex 0: [0, 1, 2, 3, 4]
BFS starting from vertex 2: [2, 0, 3, 1, 4]


### BFS Step-by-Step Example

Let's trace through BFS on a simple graph:

In [4]:
def bfs_detailed(graph: Dict[int, List[int]], start: int):
    """
    BFS with detailed step-by-step output for learning.
    """
    visited = set()
    order = []
    queue = deque([start])
    visited.add(start)
    
    print(f"Starting BFS from vertex {start}\n")
    step = 1
    
    while queue:
        print(f"Step {step}:")
        print(f"  Queue: {list(queue)}")
        print(f"  Visited so far: {sorted(visited)}")
        
        current = queue.popleft()
        order.append(current)
        print(f"  Processing vertex: {current}")
        
        neighbors = graph.get(current, [])
        print(f"  Neighbors of {current}: {neighbors}")
        
        for neighbor in neighbors:
            if neighbor not in visited:
                print(f"    -> Adding {neighbor} to queue")
                visited.add(neighbor)
                queue.append(neighbor)
        
        print()
        step += 1
    
    print(f"Final BFS order: {order}")
    return order

# Small example graph
simple_graph = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: []
}

bfs_detailed(simple_graph, 0)

Starting BFS from vertex 0

Step 1:
  Queue: [0]
  Visited so far: [0]
  Processing vertex: 0
  Neighbors of 0: [1, 2]
    -> Adding 1 to queue
    -> Adding 2 to queue

Step 2:
  Queue: [1, 2]
  Visited so far: [0, 1, 2]
  Processing vertex: 1
  Neighbors of 1: [3]
    -> Adding 3 to queue

Step 3:
  Queue: [2, 3]
  Visited so far: [0, 1, 2, 3]
  Processing vertex: 2
  Neighbors of 2: [3]

Step 4:
  Queue: [3]
  Visited so far: [0, 1, 2, 3]
  Processing vertex: 3
  Neighbors of 3: []

Final BFS order: [0, 1, 2, 3]


[0, 1, 2, 3]

## 5. Depth-First Search (DFS) <a id='dfs'></a>

**DFS** explores a graph by going as deep as possible along each branch before backtracking.

### How DFS Works:

1. Start at a source vertex and mark it as visited
2. For each unvisited neighbor:
   - Recursively visit that neighbor (go deep)
3. Backtrack when no unvisited neighbors remain

### Visual Example:

```
Graph:     0 --- 1
           |     |
           2 --- 3 --- 4
```

DFS from vertex 0 (exploring left neighbor first):
- Visit 0
- Visit 1 (first neighbor of 0)
- Visit 3 (first neighbor of 1)
- Visit 2 (first neighbor of 3, not visited yet)
- Visit 4 (backtrack to 3, then visit 4)

**Possible order**: [0, 1, 3, 2, 4] or [0, 1, 3, 4, 2] (depends on neighbor order)

### Comparison: BFS vs DFS

| Aspect | BFS | DFS |
|--------|-----|-----|
| **Data Structure** | Queue (FIFO) | Stack/Recursion (LIFO) |
| **Explores** | Level by level | Deep first |
| **Best for** | Shortest path | Connectivity, cycles |
| **Memory** | Can use more | Less for deep graphs |

### Applications:
- Finding connected components
- Detecting cycles
- Topological sorting
- Maze solving
- Path finding

### Time Complexity: O(V + E)
### Space Complexity: O(V)

In [5]:
def dfs_recursive(graph: Dict[int, List[int]], start: int, visited: Set[int] = None) -> Set[int]:
    """
    Perform Depth-First Search using recursion.
    
    Args:
        graph: Adjacency list representation
        start: Starting vertex
        visited: Set of visited vertices (for recursion)
    
    Returns:
        Set of all visited vertices
    """
    if visited is None:
        visited = set()
    
    visited.add(start)
    
    for neighbor in graph.get(start, []):
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited)
    
    return visited

# Same example graph as BFS
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2, 4],
    4: [3]
}

print("Graph:", graph)
print("\nDFS starting from vertex 0:", sorted(dfs_recursive(graph, 0)))
print("DFS starting from vertex 2:", sorted(dfs_recursive(graph, 2)))

Graph: {0: [1, 2], 1: [0, 3], 2: [0, 3], 3: [1, 2, 4], 4: [3]}

DFS starting from vertex 0: [0, 1, 2, 3, 4]
DFS starting from vertex 2: [0, 1, 2, 3, 4]


In [6]:
def dfs_iterative(graph: Dict[int, List[int]], start: int) -> List[int]:
    """
    Perform DFS using an explicit stack (iterative approach).
    Returns vertices in the order they were visited.
    """
    visited = set()
    order = []
    stack = [start]
    
    while stack:
        current = stack.pop()  # Remove from top of stack
        
        if current not in visited:
            visited.add(current)
            order.append(current)
            
            # Add neighbors to stack (in reverse order to match recursive DFS)
            for neighbor in reversed(graph.get(current, [])):
                if neighbor not in visited:
                    stack.append(neighbor)
    
    return order

print("\nDFS (Iterative) starting from vertex 0:", dfs_iterative(graph, 0))


DFS (Iterative) starting from vertex 0: [0, 1, 3, 2, 4]


### DFS Step-by-Step Example

In [7]:
def dfs_detailed(graph: Dict[int, List[int]], start: int, visited: Set[int] = None, depth: int = 0) -> Set[int]:
    """
    DFS with detailed step-by-step output for learning.
    """
    if visited is None:
        visited = set()
        print(f"Starting DFS from vertex {start}\n")
    
    indent = "  " * depth
    print(f"{indent}Visiting vertex: {start}")
    visited.add(start)
    
    neighbors = graph.get(start, [])
    print(f"{indent}Neighbors of {start}: {neighbors}")
    
    for neighbor in neighbors:
        if neighbor not in visited:
            print(f"{indent}-> Going deep to {neighbor}")
            dfs_detailed(graph, neighbor, visited, depth + 1)
            print(f"{indent}<- Backtracking from {neighbor} to {start}")
        else:
            print(f"{indent}  (Skipping {neighbor}, already visited)")
    
    return visited

# Small example graph
simple_graph = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: []
}

result = dfs_detailed(simple_graph, 0)
print(f"\nFinal visited vertices: {sorted(result)}")

Starting DFS from vertex 0

Visiting vertex: 0
Neighbors of 0: [1, 2]
-> Going deep to 1
  Visiting vertex: 1
  Neighbors of 1: [3]
  -> Going deep to 3
    Visiting vertex: 3
    Neighbors of 3: []
  <- Backtracking from 3 to 1
<- Backtracking from 1 to 0
-> Going deep to 2
  Visiting vertex: 2
  Neighbors of 2: [3]
    (Skipping 3, already visited)
<- Backtracking from 2 to 0

Final visited vertices: [0, 1, 2, 3]


## 6. Shortest Path Algorithms - SPFA <a id='spfa'></a>

### The Shortest Path Problem

Given a **weighted graph**, find the shortest path from a source vertex to all other vertices.

### SPFA: Shortest Path Faster Algorithm

SPFA is an improvement over the Bellman-Ford algorithm. It uses a queue to track vertices that need to be processed.

### How SPFA Works:

1. Initialize distances: source = 0, all others = ∞
2. Add source to queue
3. While queue is not empty:
   - Remove a vertex u from queue
   - For each neighbor v of u:
     - If distance[u] + weight(u,v) < distance[v]:
       - Update distance[v]
       - Add v to queue (if not already in queue)
4. Detect negative cycles (if a vertex is relaxed too many times)

### Key Concepts:

**Relaxation**: Updating the distance to a vertex if a shorter path is found.

```
if distance[u] + weight(u,v) < distance[v]:
    distance[v] = distance[u] + weight(u,v)
```

**Negative Cycle**: A cycle whose total edge weight is negative. If present, there's no shortest path (you can keep going around the cycle to get infinitely short paths).

### Example:

```
Graph:  0 --(5)--> 1
        |          |
       (4)       (3)
        |          |
        v          v
        2 --(6)--> 3
```

Shortest paths from 0:
- 0 → 0: 0
- 0 → 1: 5
- 0 → 2: 4
- 0 → 3: 8 (via 0→1→3)

### Time Complexity:
- Average case: O(E) - very fast in practice
- Worst case: O(V × E) - same as Bellman-Ford

### Comparison with Other Algorithms:

| Algorithm | Time | Negative Weights? | Use Case |
|-----------|------|-------------------|----------|
| **Dijkstra** | O(E log V) | ❌ No | Fastest for non-negative |
| **Bellman-Ford** | O(V × E) | ✅ Yes | Guaranteed, but slow |
| **SPFA** | O(E) avg | ✅ Yes | Fast in practice |

In [8]:
from collections import deque
from typing import Dict, List, Tuple

def spfa(graph: Dict[int, List[Tuple[int, float]]], start: int) -> Dict[int, float]:
    """
    Shortest Path Faster Algorithm (SPFA).
    
    Args:
        graph: Adjacency list with weights. Format: {node: [(neighbor, weight), ...]}
        start: Starting vertex
    
    Returns:
        Dictionary of shortest distances from start to each vertex
    
    Raises:
        ValueError: If negative weight cycle is detected
    """
    # Collect all vertices
    vertices = set(graph.keys())
    for u in graph:
        for v, _ in graph[u]:
            vertices.add(v)
    
    # Initialize distances
    distances = {v: float('inf') for v in vertices}
    distances[start] = 0
    
    # Queue for processing
    queue = deque([start])
    in_queue = {v: False for v in vertices}
    in_queue[start] = True
    
    # Count relaxations to detect negative cycles
    relax_count = {v: 0 for v in vertices}
    
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        
        for v, weight in graph.get(u, []):
            # Relaxation step
            if distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                relax_count[v] += 1
                
                # Negative cycle detection
                if relax_count[v] > len(vertices):
                    raise ValueError("Negative weight cycle detected")
                
                if not in_queue[v]:
                    queue.append(v)
                    in_queue[v] = True
    
    return distances

# Example: Graph with positive weights
weighted_graph = {
    0: [(1, 5.0), (2, 4.0)],
    1: [(3, 3.0)],
    2: [(1, 2.0), (3, 6.0)],
    3: []
}

print("Graph:", weighted_graph)
print("\nShortest distances from vertex 0:")
distances = spfa(weighted_graph, 0)
for vertex, distance in sorted(distances.items()):
    print(f"  To vertex {vertex}: {distance}")

Graph: {0: [(1, 5.0), (2, 4.0)], 1: [(3, 3.0)], 2: [(1, 2.0), (3, 6.0)], 3: []}

Shortest distances from vertex 0:
  To vertex 0: 0
  To vertex 1: 5.0
  To vertex 2: 4.0
  To vertex 3: 8.0


### SPFA with Negative Weights Example

In [9]:
# Example with negative weights (but no negative cycle)
graph_negative = {
    0: [(1, 4.0), (2, 3.0)],
    1: [(3, -2.0)],  # Negative weight!
    2: [(3, 5.0)],
    3: []
}

print("Graph with negative weights:", graph_negative)
print("\nShortest distances from vertex 0:")
try:
    distances = spfa(graph_negative, 0)
    for vertex, distance in sorted(distances.items()):
        print(f"  To vertex {vertex}: {distance}")
except ValueError as e:
    print(f"Error: {e}")

Graph with negative weights: {0: [(1, 4.0), (2, 3.0)], 1: [(3, -2.0)], 2: [(3, 5.0)], 3: []}

Shortest distances from vertex 0:
  To vertex 0: 0
  To vertex 1: 4.0
  To vertex 2: 3.0
  To vertex 3: 2.0


### Example: Detecting Negative Cycles

In [10]:
# Example with a negative cycle
graph_with_cycle = {
    0: [(1, 5.0)],
    1: [(2, 3.0)],
    2: [(3, -2.0)],
    3: [(1, -8.0)]  # Creates negative cycle: 1 -> 2 -> 3 -> 1 (total: 3 + (-2) + (-8) = -7)
}

print("Graph with negative cycle:", graph_with_cycle)
print("\nTrying to find shortest paths from vertex 0:")
try:
    distances = spfa(graph_with_cycle, 0)
    for vertex, distance in sorted(distances.items()):
        print(f"  To vertex {vertex}: {distance}")
except ValueError as e:
    print(f"❌ Error: {e}")
    print("\nExplanation: The cycle 1 → 2 → 3 → 1 has total weight -7.")
    print("Going around this cycle repeatedly makes distances infinitely small!")

Graph with negative cycle: {0: [(1, 5.0)], 1: [(2, 3.0)], 2: [(3, -2.0)], 3: [(1, -8.0)]}

Trying to find shortest paths from vertex 0:
❌ Error: Negative weight cycle detected

Explanation: The cycle 1 → 2 → 3 → 1 has total weight -7.
Going around this cycle repeatedly makes distances infinitely small!


## 7. Bellman-Ford Algorithm <a id='bellman-ford'></a>

### What is Bellman-Ford?

**Bellman-Ford** is another shortest path algorithm similar to SPFA, but it follows a more systematic approach. It relaxes all edges repeatedly and is guaranteed to find shortest paths even with negative edge weights.

### How Bellman-Ford Works:

1. Initialize distances: source = 0, all others = ∞
2. Repeat (V-1) times:
   - For each edge (u, v) with weight w:
     - If distance[u] + w < distance[v]:
       - Update distance[v] = distance[u] + w
3. One more iteration to detect negative cycles
   - If any distance can still be reduced, there's a negative cycle

### Why V-1 iterations?

In a graph with V vertices, the shortest path between any two vertices can have at most V-1 edges (without cycles). Each iteration guarantees finding shortest paths with one more edge.

### Key Differences: Bellman-Ford vs SPFA

| Feature | Bellman-Ford | SPFA |
|---------|--------------|------|
| **Approach** | Process all edges V-1 times | Only process edges from changed vertices |
| **Time (worst)** | O(V × E) guaranteed | O(V × E) worst case |
| **Time (average)** | O(V × E) | O(E) - much faster! |
| **Predictability** | Always same runtime | Depends on graph structure |
| **Best for** | Theoretical guarantees | Practical performance |

### When to Use:
- Need guaranteed O(V × E) complexity
- Academic/theoretical work
- When SPFA might degenerate (rare in practice)

In [11]:
from typing import Dict, List, Tuple

def bellman_ford(graph: Dict[int, List[Tuple[int, float]]], start: int) -> Dict[int, float]:
    """
    Bellman-Ford algorithm for shortest paths.
    
    Args:
        graph: Adjacency list with weights. Format: {node: [(neighbor, weight), ...]}
        start: Starting vertex
    
    Returns:
        Dictionary of shortest distances from start to each vertex
    
    Raises:
        ValueError: If negative weight cycle is detected
    """
    # Initialize distances
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Relax all edges V-1 times
    for _ in range(len(graph) - 1):
        for u in graph:
            for v, weight in graph[u]:
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
    
    # Check for negative cycles (one more iteration)
    for u in graph:
        for v, weight in graph[u]:
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                raise ValueError("Graph contains a negative weight cycle")
    
    return distances

# Example 1: Positive weights
graph1 = {
    0: [(1, 4.0), (2, 3.0)],
    1: [(3, 2.0)],
    2: [(3, 5.0)],
    3: []
}

print("Example 1: Graph with positive weights")
print("Graph:", graph1)
distances1 = bellman_ford(graph1, 0)
print("Shortest distances from vertex 0:", distances1)

# Example 2: With negative weights (no cycle)
graph2 = {
    0: [(1, 4.0), (2, 3.0)],
    1: [(3, -2.0)],  # Negative weight
    2: [(3, 5.0)],
    3: []
}

print("\nExample 2: Graph with negative weight (no cycle)")
print("Graph:", graph2)
distances2 = bellman_ford(graph2, 0)
print("Shortest distances from vertex 0:", distances2)
print("Notice: Path 0→1→3 (cost 2) is shorter than 0→2→3 (cost 8)!")

Example 1: Graph with positive weights
Graph: {0: [(1, 4.0), (2, 3.0)], 1: [(3, 2.0)], 2: [(3, 5.0)], 3: []}
Shortest distances from vertex 0: {0: 0, 1: 4.0, 2: 3.0, 3: 6.0}

Example 2: Graph with negative weight (no cycle)
Graph: {0: [(1, 4.0), (2, 3.0)], 1: [(3, -2.0)], 2: [(3, 5.0)], 3: []}
Shortest distances from vertex 0: {0: 0, 1: 4.0, 2: 3.0, 3: 2.0}
Notice: Path 0→1→3 (cost 2) is shorter than 0→2→3 (cost 8)!


## 8. Floyd-Warshall Algorithm <a id='floyd-warshall'></a>

### All-Pairs Shortest Path

While BFS, Bellman-Ford, and SPFA find shortest paths from **one source** to all others, **Floyd-Warshall** finds shortest paths between **all pairs** of vertices at once!

### How Floyd-Warshall Works:

The algorithm uses **dynamic programming** with a brilliant insight: 

For every pair of vertices (i, j), consider all possible intermediate vertices k. If going through k gives a shorter path, update the distance.

**Algorithm Steps:**
1. Initialize: Direct edges get their weights, others get ∞
2. For each possible intermediate vertex k:
   - For each pair of vertices (i, j):
     - If distance[i,k] + distance[k,j] < distance[i,j]:
       - Update distance[i,j]

### The Key Idea:

```
Can we go from i to j faster by going through k?

Current: i -------- j  (distance d1)
Via k:   i -> k -> j  (distance d2)

If d2 < d1, update!
```

### Example Walkthrough:

```
Initial Graph:
    0 --3--> 1
    |        |
    8        1
    |        |
    v        v
    2 --4--> 3

After considering vertex 0 as intermediate:
- Can 1→2 go faster via 0? No
- Can 2→1 go faster via 0? No
...

After considering all vertices:
All shortest paths are found!
```

### Time Complexity: O(V³)
- Three nested loops over all vertices
- Simple but powerful!

### Space Complexity: O(V²)
- Store distance for every pair

### When to Use:
- Need distances between ALL pairs of vertices
- Dense graphs where many shortest paths are needed
- Graph is small enough (V³ is manageable)

In [12]:
from typing import Dict, List, Tuple

def floyd_warshall(graph: Dict[int, List[Tuple[int, float]]]) -> Dict[Tuple[int, int], float]:
    """
    Floyd-Warshall algorithm for all-pairs shortest paths.
    
    Args:
        graph: Adjacency list with weights
    
    Returns:
        Dictionary mapping (source, dest) pairs to shortest distances
    
    Raises:
        ValueError: If negative weight cycle is detected
    """
    # Get all vertices
    vertices = list(graph.keys())
    
    # Initialize distances
    distances = {}
    for i in vertices:
        for j in vertices:
            if i == j:
                distances[(i, j)] = 0.0
            else:
                distances[(i, j)] = float('inf')
    
    # Add direct edges
    for u in vertices:
        for v, weight in graph[u]:
            distances[(u, v)] = weight
    
    # Floyd-Warshall main loop
    for k in vertices:  # Intermediate vertex
        for i in vertices:  # Source
            for j in vertices:  # Destination
                # Can we improve i→j by going through k?
                if distances[(i, k)] + distances[(k, j)] < distances[(i, j)]:
                    distances[(i, j)] = distances[(i, k)] + distances[(k, j)]
    
    # Check for negative cycles
    for v in vertices:
        if distances[(v, v)] < 0:
            raise ValueError("Graph contains a negative weight cycle")
    
    return distances

# Example graph
graph = {
    0: [(1, 3.0), (2, 8.0)],
    1: [(3, 1.0)],
    2: [(1, 4.0)],
    3: [(0, 2.0), (2, -5.0)]
}

print("Graph:", graph)
print("\nAll-pairs shortest distances:\n")

distances = floyd_warshall(graph)

# Display as a nice table
vertices = sorted(graph.keys())
print("     ", "  ".join(f"{j:3}" for j in vertices))
for i in vertices:
    row = f"{i:3}: "
    for j in vertices:
        dist = distances[(i, j)]
        if dist == float('inf'):
            row += "  ∞ "
        else:
            row += f"{dist:3.0f} "
    print(row)

print("\nExamples:")
print(f"  Shortest path from 0 to 2: {distances[(0, 2)]}")
print(f"  Shortest path from 2 to 0: {distances[(2, 0)]}")
print(f"  Note: Going 2→1→3→0 (cost 7) beats direct 2→0!")

Graph: {0: [(1, 3.0), (2, 8.0)], 1: [(3, 1.0)], 2: [(1, 4.0)], 3: [(0, 2.0), (2, -5.0)]}

All-pairs shortest distances:

        0    1    2    3
  0:   0   3  -1   4 
  1:   3   0  -4   1 
  2:   7   4   0   5 
  3:   2  -1  -5   0 

Examples:
  Shortest path from 0 to 2: -1.0
  Shortest path from 2 to 0: 7.0
  Note: Going 2→1→3→0 (cost 7) beats direct 2→0!


## 9. Minimum Spanning Trees (MST) <a id='mst'></a>

### What is a Spanning Tree?

A **spanning tree** of a graph is a subgraph that:
1. Includes all vertices
2. Is a tree (connected, no cycles)
3. Has exactly V-1 edges (where V = number of vertices)

### What is a Minimum Spanning Tree?

In a **weighted graph**, the **Minimum Spanning Tree (MST)** is the spanning tree with the smallest possible sum of edge weights.

### Real-World Applications:
- **Network Design**: Connecting cities with minimum cable/road cost
- **Cluster Analysis**: Grouping similar data points
- **Image Segmentation**: Computer vision
- **Approximation Algorithms**: For traveling salesman problem

### Example:

```
Original Graph:
    0 --10-- 1
    |   \    |
    6    5  15
    |     \  |
    2 --4-- 3

Minimum Spanning Tree (total weight = 15):
    0        1
    |        |
    6       15
    |        |
    2 --4-- 3

Better MST (total weight = 13):
    0 --5-- 1
    |       |
    6      15
    |       |
    2 --4-- 3
    
Best MST (total weight = 13):
    0 --5-- 3
    |       |
    6       4
    |       |
    2 ------+
```

Two classic algorithms to find MST:
1. **Kruskal's Algorithm** - Sort edges, add smallest edges without creating cycles
2. **Prim's Algorithm** - Grow tree from a vertex, always add cheapest edge

### 9.1 Kruskal's Algorithm

**Kruskal's approach**: Build the MST by adding edges in order of increasing weight, skipping edges that would create a cycle.

### How Kruskal's Works:

1. Sort all edges by weight (smallest first)
2. Initialize empty MST
3. For each edge (u, v) in sorted order:
   - If adding this edge doesn't create a cycle:
     - Add it to MST
   - Stop when MST has V-1 edges

### How to Detect Cycles?

Use a **Disjoint Set (Union-Find)** data structure:
- Each vertex starts in its own set
- When adding edge (u, v), check if u and v are in the same set
- If same set → would create cycle, skip
- If different sets → merge sets, add edge

### Disjoint Set Operations:
- **Find(u)**: Which set does u belong to?
- **Union(u, v)**: Merge the sets containing u and v

### Time Complexity:
- Sorting edges: O(E log E)
- Union-Find operations: Nearly O(1) with path compression
- **Total: O(E log E)** or equivalently O(E log V)

### Space Complexity: O(V) for disjoint set

In [13]:
from typing import List, Tuple

class DisjointSet:
    """Union-Find data structure for cycle detection."""
    
    def __init__(self, n: int):
        self.parent = list(range(n))  # Each node is its own parent initially
        self.rank = [0] * n  # For union by rank optimization
    
    def find(self, u: int) -> int:
        """Find the root of the set containing u (with path compression)."""
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])  # Path compression
        return self.parent[u]
    
    def union(self, u: int, v: int) -> bool:
        """Merge sets containing u and v. Returns True if merged, False if already in same set."""
        root_u = self.find(u)
        root_v = self.find(v)
        
        if root_u == root_v:
            return False  # Already in same set, would create cycle
        
        # Union by rank: attach smaller tree under larger tree
        if self.rank[root_u] > self.rank[root_v]:
            self.parent[root_v] = root_u
        elif self.rank[root_u] < self.rank[root_v]:
            self.parent[root_u] = root_v
        else:
            self.parent[root_v] = root_u
            self.rank[root_u] += 1
        
        return True

def kruskals_algorithm(num_nodes: int, edges: List[Tuple[int, int, float]]) -> List[Tuple[int, int, float]]:
    """
    Find the Minimum Spanning Tree using Kruskal's algorithm.
    
    Args:
        num_nodes: Number of vertices in the graph
        edges: List of (u, v, weight) tuples
    
    Returns:
        List of edges in the MST
    """
    # Step 1: Sort edges by weight
    edges_sorted = sorted(edges, key=lambda x: x[2])
    
    # Step 2: Initialize disjoint set and MST
    ds = DisjointSet(num_nodes)
    mst = []
    
    # Step 3: Process edges in order
    for u, v, weight in edges_sorted:
        # If u and v are in different sets, adding this edge won't create a cycle
        if ds.union(u, v):
            mst.append((u, v, weight))
            
            # Stop when we have V-1 edges
            if len(mst) == num_nodes - 1:
                break
    
    return mst

# Example graph
edges = [
    (0, 1, 10),
    (0, 2, 6),
    (0, 3, 5),
    (1, 3, 15),
    (2, 3, 4)
]
num_nodes = 4

print("All edges (u, v, weight):")
for u, v, w in sorted(edges, key=lambda x: x[2]):
    print(f"  {u} -- {v}  weight: {w}")

print("\nKruskal's Algorithm Process:")
mst = kruskals_algorithm(num_nodes, edges)

print("\nMinimum Spanning Tree edges:")
total_weight = 0
for u, v, weight in mst:
    print(f"  {u} -- {v}  weight: {weight}")
    total_weight += weight

print(f"\nTotal MST weight: {total_weight}")

All edges (u, v, weight):
  2 -- 3  weight: 4
  0 -- 3  weight: 5
  0 -- 2  weight: 6
  0 -- 1  weight: 10
  1 -- 3  weight: 15

Kruskal's Algorithm Process:

Minimum Spanning Tree edges:
  2 -- 3  weight: 4
  0 -- 3  weight: 5
  0 -- 1  weight: 10

Total MST weight: 19


### 9.2 Prim's Algorithm

**Prim's approach**: Grow the MST from a starting vertex by always adding the cheapest edge that connects a new vertex to the tree.

### How Prim's Works:

1. Start with any vertex, mark it as in the MST
2. Repeat until all vertices are in MST:
   - Find the minimum weight edge connecting a vertex in MST to a vertex outside MST
   - Add that edge and vertex to MST

### Implementation Strategy:

Use a **min-heap (priority queue)** to efficiently find the cheapest edge:
- Heap stores: (weight, vertex, parent)
- Always extract minimum weight edge
- Add neighbors of newly added vertex to heap

### Time Complexity:
- With binary heap: O(E log V)
- With Fibonacci heap: O(E + V log V)

### Space Complexity: O(V) for visited array and heap

### Kruskal vs Prim:

| Feature | Kruskal | Prim |
|---------|---------|------|
| **Approach** | Edge-based | Vertex-based |
| **Data Structure** | Disjoint Set | Priority Queue |
| **Best for** | Sparse graphs | Dense graphs |
| **Parallelizable?** | Yes (sort edges) | No (sequential growth) |
| **Easier to understand?** | Yes | Slightly more complex |

In [14]:
import heapq
from typing import List, Tuple, Dict

def prims_algorithm(num_nodes: int, edges: List[Tuple[int, int, float]]) -> List[Tuple[int, int, float]]:
    """
    Find the Minimum Spanning Tree using Prim's algorithm.
    
    Args:
        num_nodes: Number of vertices in the graph
        edges: List of (u, v, weight) tuples
    
    Returns:
        List of edges in the MST
    """
    # Build adjacency list
    graph = {i: [] for i in range(num_nodes)}
    for u, v, weight in edges:
        graph[u].append((weight, v))
        graph[v].append((weight, u))
    
    mst = []
    visited = [False] * num_nodes
    
    # Start from vertex 0
    # Min-heap: (weight, current_vertex, parent_vertex)
    min_heap = [(0, 0, -1)]
    
    while min_heap:
        weight, current, parent = heapq.heappop(min_heap)
        
        # Skip if already visited
        if visited[current]:
            continue
        
        # Mark as visited
        visited[current] = True
        
        # Add edge to MST (except for the starting vertex)
        if parent != -1:
            mst.append((parent, current, weight))
        
        # Add all edges from current vertex to unvisited neighbors
        for edge_weight, neighbor in graph[current]:
            if not visited[neighbor]:
                heapq.heappush(min_heap, (edge_weight, neighbor, current))
    
    return mst

# Same example as Kruskal's
edges = [
    (0, 1, 10),
    (0, 2, 6),
    (0, 3, 5),
    (1, 3, 15),
    (2, 3, 4)
]
num_nodes = 4

print("Graph edges:")
for u, v, w in edges:
    print(f"  {u} -- {v}  weight: {w}")

print("\nPrim's Algorithm Process (starting from vertex 0):")
mst = prims_algorithm(num_nodes, edges)

print("\nMinimum Spanning Tree edges:")
total_weight = 0
for u, v, weight in mst:
    print(f"  {u} -- {v}  weight: {weight}")
    total_weight += weight

print(f"\nTotal MST weight: {total_weight}")
print("\nNote: Same result as Kruskal's! MST is unique if all edge weights are distinct.")

Graph edges:
  0 -- 1  weight: 10
  0 -- 2  weight: 6
  0 -- 3  weight: 5
  1 -- 3  weight: 15
  2 -- 3  weight: 4

Prim's Algorithm Process (starting from vertex 0):

Minimum Spanning Tree edges:
  0 -- 3  weight: 5
  3 -- 2  weight: 4
  0 -- 1  weight: 10

Total MST weight: 19

Note: Same result as Kruskal's! MST is unique if all edge weights are distinct.

  0 -- 1  weight: 10
  0 -- 2  weight: 6
  0 -- 3  weight: 5
  1 -- 3  weight: 15
  2 -- 3  weight: 4

Prim's Algorithm Process (starting from vertex 0):

Minimum Spanning Tree edges:
  0 -- 3  weight: 5
  3 -- 2  weight: 4
  0 -- 1  weight: 10

Total MST weight: 19

Note: Same result as Kruskal's! MST is unique if all edge weights are distinct.


## 10. Eulerian Paths and Circuits <a id='eulerian'></a>

### What is an Eulerian Path?

An **Eulerian path** is a trail in a graph that visits **every edge exactly once**.

An **Eulerian circuit** is an Eulerian path that starts and ends at the same vertex.

### Historical Background:

The famous **Seven Bridges of Königsberg** problem (1736):
- City had 7 bridges connecting 4 land areas
- Question: Can you walk crossing each bridge exactly once?
- Leonhard Euler proved it's impossible!
- This created graph theory as a field

### Conditions for Eulerian Paths:

**For Undirected Graphs:**

1. **Eulerian Circuit exists** if:
   - Graph is connected
   - ALL vertices have **even degree**

2. **Eulerian Path exists** (not circuit) if:
   - Graph is connected  
   - Exactly **2 vertices have odd degree**
   - Path must start and end at these odd-degree vertices

3. **No Eulerian Path** if:
   - More than 2 vertices have odd degree

### Why does this work?

Think about entering and leaving vertices:
- Each time you pass through a vertex, you use 2 edges (one in, one out)
- To visit all edges, you need to pair them up
- Odd degree means one edge is unpaired → must be start or end

### Real-World Applications:
- **Route planning**: Mail delivery, street cleaning
- **Circuit design**: Drawing circuits without lifting pen
- **DNA sequencing**: Reconstructing DNA from fragments
- **Network verification**: Testing network connections

In [15]:
from typing import List, Tuple
from collections import defaultdict, deque

def has_eulerian_path(num_nodes: int, edges: List[Tuple[int, int]]) -> bool:
    """Check if the graph has an Eulerian path."""
    # Calculate degree of each vertex
    degree = [0] * num_nodes
    for u, v in edges:
        degree[u] += 1
        degree[v] += 1
    
    # Count vertices with odd degree
    odd_count = sum(1 for deg in degree if deg % 2 != 0)
    
    # Eulerian path exists if 0 or 2 vertices have odd degree
    return odd_count == 0 or odd_count == 2

def find_eulerian_path(num_nodes: int, edges: List[Tuple[int, int]]) -> List[int]:
    """
    Find an Eulerian path using Hierholzer's algorithm.
    
    Returns the path as a list of vertices, or empty list if no path exists.
    """
    if not has_eulerian_path(num_nodes, edges):
        return []
    
    # Build adjacency list (with edge removal capability)
    graph = defaultdict(deque)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
    
    # Find starting vertex (vertex with odd degree, or any vertex)
    start_node = 0
    for i in range(num_nodes):
        if len(graph[i]) % 2 != 0:
            start_node = i
            break
    
    # Hierholzer's algorithm
    path = []
    stack = [start_node]
    
    while stack:
        u = stack[-1]
        if graph[u]:
            # Take an edge
            v = graph[u].popleft()
            graph[v].remove(u)  # Remove reverse edge
            stack.append(v)
        else:
            # No more edges from u, add to path
            path.append(stack.pop())
    
    return path[::-1]  # Reverse to get correct order

# Example 1: Graph with Eulerian Circuit (all even degrees)
print("Example 1: Eulerian Circuit (Triangle)")
edges1 = [(0, 1), (1, 2), (2, 0)]
num_nodes1 = 3

print("Edges:", edges1)
degrees1 = [0] * num_nodes1
for u, v in edges1:
    degrees1[u] += 1
    degrees1[v] += 1
print("Degrees:", degrees1)

if has_eulerian_path(num_nodes1, edges1):
    path1 = find_eulerian_path(num_nodes1, edges1)
    print("Eulerian path found:", path1)
    print("(This is actually a circuit - starts and ends at same vertex)\n")

# Example 2: Graph with Eulerian Path (exactly 2 odd degrees)
print("Example 2: Eulerian Path (Not a circuit)")
edges2 = [
    (0, 1),
    (1, 2),
    (2, 0),
    (1, 3),
    (3, 4),
    (4, 1)
]
num_nodes2 = 5

print("Edges:", edges2)
degrees2 = [0] * num_nodes2
for u, v in edges2:
    degrees2[u] += 1
    degrees2[v] += 1
print("Degrees:", degrees2)
print("Vertices with odd degree:", [i for i, d in enumerate(degrees2) if d % 2 != 0])

if has_eulerian_path(num_nodes2, edges2):
    path2 = find_eulerian_path(num_nodes2, edges2)
    print("Eulerian path found:", path2)
    print("Notice: Path starts at vertex with odd degree\n")

# Example 3: No Eulerian Path (more than 2 odd degrees)
print("Example 3: No Eulerian Path")
edges3 = [(0, 1), (1, 2), (2, 3)]
num_nodes3 = 4

print("Edges:", edges3)
degrees3 = [0] * num_nodes3
for u, v in edges3:
    degrees3[u] += 1
    degrees3[v] += 1
print("Degrees:", degrees3)
print("Vertices with odd degree:", [i for i, d in enumerate(degrees3) if d % 2 != 0])

if has_eulerian_path(num_nodes3, edges3):
    path3 = find_eulerian_path(num_nodes3, edges3)
    print("Eulerian path found:", path3)
else:
    print("No Eulerian path exists (4 vertices with odd degree)")

Example 1: Eulerian Circuit (Triangle)
Edges: [(0, 1), (1, 2), (2, 0)]
Degrees: [2, 2, 2]
Eulerian path found: [0, 1, 2, 0]
(This is actually a circuit - starts and ends at same vertex)

Example 2: Eulerian Path (Not a circuit)
Edges: [(0, 1), (1, 2), (2, 0), (1, 3), (3, 4), (4, 1)]
Degrees: [2, 4, 2, 2, 2]
Vertices with odd degree: []
Eulerian path found: [0, 1, 3, 4, 1, 2, 0]
Notice: Path starts at vertex with odd degree

Example 3: No Eulerian Path
Edges: [(0, 1), (1, 2), (2, 3)]
Degrees: [1, 2, 2, 1]
Vertices with odd degree: [0, 3]
Eulerian path found: [0, 1, 2, 3]


## 11. Hamiltonian Paths and Circuits <a id='hamiltonian'></a>

### What is a Hamiltonian Path?

A **Hamiltonian path** is a path in a graph that visits **every vertex exactly once**.

A **Hamiltonian circuit** is a Hamiltonian path that returns to the starting vertex (forming a cycle).

### Eulerian vs Hamiltonian:

| Feature | Eulerian | Hamiltonian |
|---------|----------|-------------|
| **Visits** | Every **edge** once | Every **vertex** once |
| **Condition** | Simple (degree check) | No simple condition! |
| **Detection** | O(V + E) - polynomial | NP-Complete - hard! |
| **Real Example** | Mail delivery route | Traveling salesman |

### The Difficulty:

Unlike Eulerian paths (which have a simple degree check), there's **no known efficient algorithm** to check if a Hamiltonian path exists!

This is one of the famous **NP-Complete problems**:
- Easy to verify a solution
- Hard to find a solution
- No known polynomial-time algorithm

### Why is it Hard?

For a graph with n vertices:
- Potential paths: n! (factorial) possibilities
- Example: 10 vertices → 3,628,800 paths to check!
- 20 vertices → 2,432,902,008,176,640,000 paths!

### Approach: Backtracking

Since there's no efficient algorithm, we use **backtracking**:
1. Try to build a path vertex by vertex
2. If stuck, backtrack and try different choices
3. Explore all possibilities until finding a path (or proving none exists)

### Real-World Applications:
- **Traveling Salesman Problem**: Visit all cities with minimum cost
- **Circuit Board Design**: Optimal drilling paths
- **DNA Sequencing**: Ordering fragments
- **Job Scheduling**: Complete all tasks with dependencies

In [16]:
from typing import List, Tuple, Set
from collections import defaultdict

def find_hamiltonian_path(num_nodes: int, edges: List[Tuple[int, int]]) -> List[int]:
    """
    Find a Hamiltonian path using backtracking.
    
    Returns the path as a list of vertices, or empty list if no path exists.
    
    WARNING: This can be very slow for large graphs!
    """
    # Build adjacency list
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
    
    def backtrack(path: List[int], visited: Set[int]) -> bool:
        """Try to extend the path to visit all vertices."""
        # Base case: visited all vertices
        if len(path) == num_nodes:
            return True
        
        # Try to extend path from last vertex
        last = path[-1]
        for neighbor in graph[last]:
            if neighbor not in visited:
                # Try this neighbor
                path.append(neighbor)
                visited.add(neighbor)
                
                if backtrack(path, visited):
                    return True
                
                # Backtrack
                path.pop()
                visited.remove(neighbor)
        
        return False
    
    # Try starting from each vertex
    for start in range(num_nodes):
        path = [start]
        visited = {start}
        if backtrack(path, visited):
            return path
    
    return []

# Example 1: Square with diagonal - has Hamiltonian path
print("Example 1: Square with one diagonal")
edges1 = [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 0),
    (0, 2)  # Diagonal
]
num_nodes1 = 4

print("Edges:", edges1)
print("Adjacency list:")
graph1 = defaultdict(list)
for u, v in edges1:
    graph1[u].append(v)
    graph1[v].append(u)
for node in sorted(graph1.keys()):
    print(f"  {node}: {graph1[node]}")

path1 = find_hamiltonian_path(num_nodes1, edges1)
if path1:
    print(f"Hamiltonian path found: {path1}")
    print(f"Verifying: visits all {num_nodes1} vertices ✓\n")
else:
    print("No Hamiltonian path exists\n")

# Example 2: Simple chain - has Hamiltonian path
print("Example 2: Simple chain (0-1-2-3)")
edges2 = [(0, 1), (1, 2), (2, 3)]
num_nodes2 = 4

print("Edges:", edges2)
path2 = find_hamiltonian_path(num_nodes2, edges2)
if path2:
    print(f"Hamiltonian path found: {path2}")
else:
    print("No Hamiltonian path exists")

print("\n" + "="*60)
print("Important Note:")
print("="*60)
print("For large graphs (>15 vertices), this algorithm can take")
print("a VERY long time due to the exponential search space!")
print("In practice, heuristics and approximations are used.")

Example 1: Square with one diagonal
Edges: [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]
Adjacency list:
  0: [1, 3, 2]
  1: [0, 2]
  2: [1, 3, 0]
  3: [2, 0]
Hamiltonian path found: [0, 1, 2, 3]
Verifying: visits all 4 vertices ✓

Example 2: Simple chain (0-1-2-3)
Edges: [(0, 1), (1, 2), (2, 3)]
Hamiltonian path found: [0, 1, 2, 3]

Important Note:
For large graphs (>15 vertices), this algorithm can take
a VERY long time due to the exponential search space!
In practice, heuristics and approximations are used.

Edges: [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]
Adjacency list:
  0: [1, 3, 2]
  1: [0, 2]
  2: [1, 3, 0]
  3: [2, 0]
Hamiltonian path found: [0, 1, 2, 3]
Verifying: visits all 4 vertices ✓

Example 2: Simple chain (0-1-2-3)
Edges: [(0, 1), (1, 2), (2, 3)]
Hamiltonian path found: [0, 1, 2, 3]

Important Note:
For large graphs (>15 vertices), this algorithm can take
a VERY long time due to the exponential search space!
In practice, heuristics and approximations are used.


## 12. Knight's Tour Problem <a id='knights-tour'></a>

### The Classic Chess Problem

The **Knight's Tour** is a special case of the Hamiltonian path problem on a chessboard.

**Goal**: Move a chess knight so it visits every square on an n×n board exactly once.

### Knight Moves:

A knight moves in an "L" shape:
- 2 squares in one direction
- 1 square perpendicular

From any square, a knight has up to 8 possible moves:
```
  . . X . X . .
  . X . . . X .
  . . . K . . .
  . X . . . X .
  . . X . X . .
```

Where K = knight, X = possible moves

### Why is this Hard?

- An 8×8 board has 64 squares
- This is a Hamiltonian path on a graph with 64 vertices!
- Naive backtracking would take forever

### Warnsdorf's Rule (Heuristic):

A smart optimization for backtracking:

**Always move to the square from which the knight will have the fewest onward moves.**

Intuition: 
- Visit "hard to reach" squares early
- Leaves more options for later moves
- Dramatically reduces backtracking

### Example (5×5 board):

```
Start at (0,0), mark as 1
Move to square with fewest onward moves
Continue until all squares visited

Result:
 1 10 15 20  3
14 19  2 11 16
 9 24 13 18 21
25 12  7  4 23
 6  8 22 17  5
```

### Time Complexity:
- Worst case: Still exponential
- With Warnsdorf's rule: Much faster in practice
- Can solve 8×8 board in reasonable time

In [17]:
from typing import List, Tuple, Optional

def knights_tour(n: int, start_x: int = 0, start_y: int = 0) -> Optional[List[Tuple[int, int]]]:
    """
    Find a Knight's Tour on an n×n chessboard using Warnsdorf's rule.
    
    Args:
        n: Size of the board (n×n)
        start_x, start_y: Starting position
    
    Returns:
        List of (x, y) positions in order, or None if no tour exists
    """
    # All possible knight moves (8 L-shaped moves)
    moves = [
        (2, 1), (1, 2), (-1, 2), (-2, 1),
        (-2, -1), (-1, -2), (1, -2), (2, -1)
    ]
    
    def is_valid(x: int, y: int, visited: List[List[bool]]) -> bool:
        """Check if position is valid and unvisited."""
        return 0 <= x < n and 0 <= y < n and not visited[x][y]
    
    def count_onward_moves(x: int, y: int, visited: List[List[bool]]) -> int:
        """Count how many valid moves from (x, y) - Warnsdorf's rule."""
        count = 0
        for dx, dy in moves:
            if is_valid(x + dx, y + dy, visited):
                count += 1
        return count
    
    def backtrack(x: int, y: int, move_count: int, 
                  path: List[Tuple[int, int]], visited: List[List[bool]]) -> bool:
        """Try to complete the tour using backtracking with Warnsdorf's rule."""
        if move_count == n * n:
            return True  # Tour complete!
        
        # Get all possible next moves with their onward move counts
        next_moves = []
        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny, visited):
                onward = count_onward_moves(nx, ny, visited)
                next_moves.append((onward, nx, ny))
        
        # Sort by number of onward moves (Warnsdorf's rule)
        # Try squares with fewer onward moves first
        next_moves.sort()
        
        for _, nx, ny in next_moves:
            visited[nx][ny] = True
            path.append((nx, ny))
            
            if backtrack(nx, ny, move_count + 1, path, visited):
                return True
            
            # Backtrack
            visited[nx][ny] = False
            path.pop()
        
        return False
    
    # Initialize
    visited = [[False] * n for _ in range(n)]
    visited[start_x][start_y] = True
    path = [(start_x, start_y)]
    
    if backtrack(start_x, start_y, 1, path, visited):
        return path
    else:
        return None

# Example: 5×5 board
n = 5
print(f"Finding Knight's Tour on a {n}×{n} board...\n")

tour = knights_tour(n)

if tour:
    print("Knight's Tour found! ✓")
    print("\nPath sequence:")
    for i in range(0, len(tour), 10):
        print("  ", tour[i:i+10])
    
    # Display as a board
    print(f"\nBoard (showing move order):")
    board = [[0] * n for _ in range(n)]
    for move_num, (x, y) in enumerate(tour, start=1):
        board[x][y] = move_num
    
    for row in board:
        print("  ", ' '.join(f"{cell:2}" for cell in row))
    
    print(f"\n✓ All {n*n} squares visited exactly once!")
else:
    print("No Knight's Tour found for this configuration.")

print("\n" + "="*60)
print("Try different board sizes:")
print("  n=5: Very fast")
print("  n=6: Usually fast") 
print("  n=7: Can take longer")
print("  n=8: May take a while without Warnsdorf's rule")

Finding Knight's Tour on a 5×5 board...

Knight's Tour found! ✓

Path sequence:
   [(0, 0), (1, 2), (0, 4), (2, 3), (4, 4), (3, 2), (4, 0), (2, 1), (0, 2), (1, 0)]
   [(3, 1), (4, 3), (2, 4), (0, 3), (1, 1), (3, 0), (4, 2), (3, 4), (1, 3), (0, 1)]
   [(2, 0), (4, 1), (2, 2), (1, 4), (3, 3)]

Board (showing move order):
    1 20  9 14  3
   10 15  2 19 24
   21  8 23  4 13
   16 11  6 25 18
    7 22 17 12  5

✓ All 25 squares visited exactly once!

Try different board sizes:
  n=5: Very fast
  n=6: Usually fast
  n=7: Can take longer
  n=8: May take a while without Warnsdorf's rule

Knight's Tour found! ✓

Path sequence:
   [(0, 0), (1, 2), (0, 4), (2, 3), (4, 4), (3, 2), (4, 0), (2, 1), (0, 2), (1, 0)]
   [(3, 1), (4, 3), (2, 4), (0, 3), (1, 1), (3, 0), (4, 2), (3, 4), (1, 3), (0, 1)]
   [(2, 0), (4, 1), (2, 2), (1, 4), (3, 3)]

Board (showing move order):
    1 20  9 14  3
   10 15  2 19 24
   21  8 23  4 13
   16 11  6 25 18
    7 22 17 12  5

✓ All 25 squares visited exactly once!



## 13. Binary Trees <a id='binary-trees'></a>

### What is a Binary Tree?

A **binary tree** is a special type of tree where each node has at most **two children**: a left child and a right child.

### Structure:
```
       1
      / \
     2   3
    / \
   4   5
      /
     6
        \
         7
```

### Tree Terminology:
- **Root**: Top node (1 in example)
- **Parent**: Node with children
- **Child**: Node descended from another
- **Leaf**: Node with no children (4, 6, 7, 3)
- **Subtree**: Tree formed by a node and its descendants
- **Height**: Longest path from root to leaf
- **Depth**: Distance from root to a node

### Binary Tree vs General Tree:

| Feature | Binary Tree | General Tree |
|---------|-------------|--------------|
| **Max children** | 2 per node | Unlimited |
| **Child distinction** | Left vs Right matters | No order distinction |
| **Empty subtrees** | Can have empty left/right | N/A |

### Tree Traversals

Three main ways to traverse a binary tree (visit all nodes):

**1. Pre-order**: Root → Left → Right
   - Visit root first
   - Then left subtree
   - Then right subtree
   - Example: [1, 2, 4, 5, 6, 7, 3]

**2. In-order**: Left → Root → Right
   - Visit left subtree
   - Then root
   - Then right subtree  
   - Example: [4, 2, 6, 5, 7, 1, 3]
   - Special: For binary search trees, gives sorted order!

**3. Post-order**: Left → Right → Root
   - Visit left subtree
   - Then right subtree
   - Then root
   - Example: [4, 6, 7, 5, 2, 3, 1]

### Reconstructing Trees

Given two traversals, can we reconstruct the original tree?

| Given Traversals | Can Reconstruct? |
|-----------------|------------------|
| Pre-order + In-order | ✅ Yes, uniquely |
| Post-order + In-order | ✅ Yes, uniquely |
| Pre-order + Post-order | ❌ No (ambiguous) |

**Why?** In-order tells us which nodes are left/right of root!

In [18]:
from typing import List, Optional

class BinaryTreeNode:
    """Node in a binary tree."""
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def build_tree_from_pre_in(preorder: List[int], inorder: List[int]) -> Optional[BinaryTreeNode]:
    """
    Reconstruct binary tree from pre-order and in-order traversals.
    
    How it works:
    - First element of preorder is the root
    - Find root in inorder to split left/right subtrees
    - Recursively build left and right subtrees
    """
    if not preorder or not inorder:
        return None
    
    # Root is first in pre-order
    root_value = preorder[0]
    root = BinaryTreeNode(root_value)
    
    # Find root position in in-order
    root_index = inorder.index(root_value)
    
    # Elements before root in in-order are in left subtree
    # Elements after root in in-order are in right subtree
    
    root.left = build_tree_from_pre_in(
        preorder[1:1 + root_index],
        inorder[:root_index]
    )
    
    root.right = build_tree_from_pre_in(
        preorder[1 + root_index:],
        inorder[root_index + 1:]
    )
    
    return root

def build_tree_from_post_in(postorder: List[int], inorder: List[int]) -> Optional[BinaryTreeNode]:
    """
    Reconstruct binary tree from post-order and in-order traversals.
    
    How it works:
    - Last element of postorder is the root
    - Find root in inorder to split left/right subtrees
    - Recursively build left and right subtrees
    """
    if not postorder or not inorder:
        return None
    
    # Root is last in post-order
    root_value = postorder[-1]
    root = BinaryTreeNode(root_value)
    
    # Find root position in in-order
    root_index = inorder.index(root_value)
    
    root.left = build_tree_from_post_in(
        postorder[:root_index],
        inorder[:root_index]
    )
    
    root.right = build_tree_from_post_in(
        postorder[root_index:-1],
        inorder[root_index + 1:]
    )
    
    return root

def print_tree(node: Optional[BinaryTreeNode], level: int = 0, prefix: str = "Root: "):
    """Print tree structure."""
    if node is not None:
        print("  " * level + prefix + str(node.value))
        if node.left or node.right:
            if node.left:
                print_tree(node.left, level + 1, "L--- ")
            else:
                print("  " * (level + 1) + "L--- (empty)")
            if node.right:
                print_tree(node.right, level + 1, "R--- ")
            else:
                print("  " * (level + 1) + "R--- (empty)")

def get_preorder(node: Optional[BinaryTreeNode]) -> List[int]:
    """Get pre-order traversal."""
    if not node:
        return []
    return [node.value] + get_preorder(node.left) + get_preorder(node.right)

def get_inorder(node: Optional[BinaryTreeNode]) -> List[int]:
    """Get in-order traversal."""
    if not node:
        return []
    return get_inorder(node.left) + [node.value] + get_inorder(node.right)

def get_postorder(node: Optional[BinaryTreeNode]) -> List[int]:
    """Get post-order traversal."""
    if not node:
        return []
    return get_postorder(node.left) + get_postorder(node.right) + [node.value]

# Example: Build tree from traversals
print("Given traversals:")
preorder =  [1, 2, 4, 5, 6, 7, 3]
inorder =   [4, 2, 6, 5, 7, 1, 3]
postorder = [4, 6, 7, 5, 2, 3, 1]

print(f"  Pre-order:  {preorder}")
print(f"  In-order:   {inorder}")
print(f"  Post-order: {postorder}")

# Reconstruct from pre-order + in-order
print("\n1. Reconstructing from pre-order + in-order:")
tree1 = build_tree_from_pre_in(preorder, inorder)
print_tree(tree1)

# Verify by generating traversals
print("\nVerification:")
print(f"  Pre-order:  {get_preorder(tree1)} ✓")
print(f"  In-order:   {get_inorder(tree1)} ✓")
print(f"  Post-order: {get_postorder(tree1)} ✓")

# Reconstruct from post-order + in-order
print("\n2. Reconstructing from post-order + in-order:")
tree2 = build_tree_from_post_in(postorder, inorder)
print("  (Should produce the same tree)")
print(f"  Pre-order:  {get_preorder(tree2)} ✓")

Given traversals:
  Pre-order:  [1, 2, 4, 5, 6, 7, 3]
  In-order:   [4, 2, 6, 5, 7, 1, 3]
  Post-order: [4, 6, 7, 5, 2, 3, 1]

1. Reconstructing from pre-order + in-order:
Root: 1
  L--- 2
    L--- 4
    R--- 5
      L--- 6
      R--- 7
  R--- 3

Verification:
  Pre-order:  [1, 2, 4, 5, 6, 7, 3] ✓
  In-order:   [4, 2, 6, 5, 7, 1, 3] ✓
  Post-order: [4, 6, 7, 5, 2, 3, 1] ✓

2. Reconstructing from post-order + in-order:
  (Should produce the same tree)
  Pre-order:  [1, 2, 4, 5, 6, 7, 3] ✓


## 7. Practical Examples <a id='examples'></a>

Let's apply what we've learned to solve real-world problems.

### Example 1: Social Network - Finding Degrees of Connection

Problem: In a social network, find the minimum number of connections between two people.

In [19]:
def degrees_of_separation(graph: Dict[str, List[str]], person1: str, person2: str) -> int:
    """
    Find the minimum degrees of separation between two people.
    Uses BFS to find shortest path.
    """
    if person1 == person2:
        return 0
    
    visited = {person1}
    queue = deque([(person1, 0)])  # (person, distance)
    
    while queue:
        current, distance = queue.popleft()
        
        for friend in graph.get(current, []):
            if friend == person2:
                return distance + 1
            
            if friend not in visited:
                visited.add(friend)
                queue.append((friend, distance + 1))
    
    return -1  # Not connected

# Social network example
social_network = {
    'Alice': ['Bob', 'Carol'],
    'Bob': ['Alice', 'David', 'Eve'],
    'Carol': ['Alice', 'Frank'],
    'David': ['Bob'],
    'Eve': ['Bob', 'Frank'],
    'Frank': ['Carol', 'Eve', 'Grace'],
    'Grace': ['Frank']
}

print("Social Network:")
for person, friends in social_network.items():
    print(f"  {person} is friends with: {', '.join(friends)}")

print("\nDegrees of separation:")
print(f"  Alice to David: {degrees_of_separation(social_network, 'Alice', 'David')} degrees")
print(f"  Alice to Grace: {degrees_of_separation(social_network, 'Alice', 'Grace')} degrees")
print(f"  Bob to Frank: {degrees_of_separation(social_network, 'Bob', 'Frank')} degrees")

Social Network:
  Alice is friends with: Bob, Carol
  Bob is friends with: Alice, David, Eve
  Carol is friends with: Alice, Frank
  David is friends with: Bob
  Eve is friends with: Bob, Frank
  Frank is friends with: Carol, Eve, Grace
  Grace is friends with: Frank

Degrees of separation:
  Alice to David: 2 degrees
  Alice to Grace: 3 degrees
  Bob to Frank: 2 degrees


### Example 2: City Navigation - Finding Cheapest Route

Problem: Find the cheapest route between cities.

In [20]:
# City transportation network with costs
city_network = {
    'NYC': [('Boston', 50), ('Philadelphia', 30)],
    'Boston': [('NYC', 50), ('Portland', 80)],
    'Philadelphia': [('NYC', 30), ('Washington', 40), ('Pittsburgh', 100)],
    'Washington': [('Philadelphia', 40), ('Richmond', 60)],
    'Pittsburgh': [('Philadelphia', 100), ('Cleveland', 70)],
    'Cleveland': [('Pittsburgh', 70)],
    'Portland': [('Boston', 80)],
    'Richmond': [('Washington', 60)]
}

print("City Network:")
for city, routes in city_network.items():
    print(f"  From {city}: {routes}")

print("\nCheapest routes from NYC:")
try:
    costs = spfa(city_network, 'NYC')
    for city, cost in sorted(costs.items()):
        if cost == float('inf'):
            print(f"  To {city}: Not reachable")
        else:
            print(f"  To {city}: ${cost}")
except ValueError as e:
    print(f"Error: {e}")

City Network:
  From NYC: [('Boston', 50), ('Philadelphia', 30)]
  From Boston: [('NYC', 50), ('Portland', 80)]
  From Philadelphia: [('NYC', 30), ('Washington', 40), ('Pittsburgh', 100)]
  From Washington: [('Philadelphia', 40), ('Richmond', 60)]
  From Pittsburgh: [('Philadelphia', 100), ('Cleveland', 70)]
  From Cleveland: [('Pittsburgh', 70)]
  From Portland: [('Boston', 80)]
  From Richmond: [('Washington', 60)]

Cheapest routes from NYC:
  To Boston: $50
  To Cleveland: $200
  To NYC: $0
  To Philadelphia: $30
  To Pittsburgh: $130
  To Portland: $130
  To Richmond: $130
  To Washington: $70


### Example 3: Connected Components

Problem: Find all separate groups in a network (e.g., friend circles, isolated networks).

In [21]:
def find_connected_components(graph: Dict[int, List[int]]) -> List[Set[int]]:
    """
    Find all connected components in an undirected graph.
    Uses DFS to explore each component.
    """
    visited = set()
    components = []
    
    # Get all vertices
    all_vertices = set(graph.keys())
    for neighbors in graph.values():
        all_vertices.update(neighbors)
    
    for vertex in all_vertices:
        if vertex not in visited:
            # Start a new component
            component = set()
            
            # DFS to find all vertices in this component
            stack = [vertex]
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    component.add(current)
                    stack.extend(graph.get(current, []))
            
            components.append(component)
    
    return components

# Graph with multiple disconnected components
disconnected_graph = {
    0: [1, 2],
    1: [0],
    2: [0],
    3: [4, 5],
    4: [3],
    5: [3],
    6: [7],
    7: [6],
    8: []  # Isolated vertex
}

print("Graph:", disconnected_graph)
print("\nConnected components:")
components = find_connected_components(disconnected_graph)
for i, component in enumerate(components, 1):
    print(f"  Component {i}: {sorted(component)}")

Graph: {0: [1, 2], 1: [0], 2: [0], 3: [4, 5], 4: [3], 5: [3], 6: [7], 7: [6], 8: []}

Connected components:
  Component 1: [0, 1, 2]
  Component 2: [3, 4, 5]
  Component 3: [6, 7]
  Component 4: [8]


## Summary

### Key Takeaways:

1. **Graphs** model relationships: vertices (objects) connected by edges (relationships)

2. **Representations**:
   - Adjacency List: Memory-efficient, good for sparse graphs
   - Adjacency Matrix: Fast edge lookup, good for dense graphs

3. **Traversal Algorithms**:
   - **BFS**: Level by level, shortest path in unweighted graphs
   - **DFS**: Deep first, good for connectivity and cycles

4. **Shortest Path**:
   - **SPFA**: Efficient for weighted graphs, handles negative weights
   - Detects negative cycles

### When to Use What?

- **Need shortest path (unweighted)?** → BFS
- **Need to explore all vertices?** → DFS or BFS
- **Need shortest path (weighted)?** → SPFA or Dijkstra
- **Have negative weights?** → SPFA or Bellman-Ford
- **Need to find cycles?** → DFS
- **Need to find connected components?** → DFS

### Practice Tips:

1. Start with small graphs and trace through algorithms manually
2. Draw graphs on paper to visualize the problem
3. Implement the basic algorithms from scratch
4. Solve problems on LeetCode, HackerRank, or Codeforces
5. Think about real-world applications

## Further Reading and Resources

### Books:
- "Introduction to Algorithms" by Cormen, Leiserson, Rivest, and Stein (CLRS)
- "Algorithms" by Robert Sedgewick and Kevin Wayne
- "Competitive Programmer's Handbook" by Antti Laaksonen

### Online Resources:
- [Visualgo](https://visualgo.net/) - Algorithm visualizations
- [Graph Theory by William Fiset (YouTube)](https://www.youtube.com/watch?v=09_LlHjoEiY)
- [LeetCode Graph Problems](https://leetcode.com/tag/graph/)

### Practice Problems:
1. Find if path exists between two vertices
2. Count number of islands (2D grid)
3. Course schedule (topological sort)
4. Network delay time (shortest path)
5. Minimum spanning tree problems